In [ ]:
!git add .

In [ ]:
!git commit -m "Descrição do problema, criação do notebook e dataset"

In [ ]:
!git push origin main

## The problem:

Uma empresa de seguros de saúde precisa detectar os clientes que são mais propensos a comprar um seguro de automóvel. 

O desafio é calcular a propensão de compra de cada cliente da base e retornar para o time de negócio, uma lista priorizada, começando pelo cliente mais propenso até o menor propenso.

# 0.0. Imports

## 0.1. lib imports

In [12]:
from acessos import database_name, host, port, user

In [10]:
print('dsfdsf')

dsfdsf
